In [1]:
# Python 2 & 3 Compatibility
from __future__ import print_function, division

# Necessary imports
import pandas as pd

import os
import pickle
from datetime import datetime
%matplotlib inline


In [2]:
df_ad = pd.read_csv('data/train.csv')
df_ad

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,83230,3,1,13,379,2017-11-06 14:32:21,NaN,0
1,17357,3,1,19,379,2017-11-06 14:33:34,NaN,0
2,35810,3,1,13,379,2017-11-06 14:34:12,NaN,0
3,45745,14,1,13,478,2017-11-06 14:34:52,NaN,0
4,161007,3,1,13,379,2017-11-06 14:35:08,NaN,0
5,18787,3,1,16,379,2017-11-06 14:36:26,NaN,0
6,103022,3,1,23,379,2017-11-06 14:37:44,NaN,0
7,114221,3,1,19,379,2017-11-06 14:37:59,NaN,0
8,165970,3,1,13,379,2017-11-06 14:38:10,NaN,0
9,74544,64,1,22,459,2017-11-06 14:38:23,NaN,0


In [3]:
def convertToCategoryType(cols, old_df):
    for v in cols:
        old_df[v] = old_df[v].astype('category')
    return old_df;


In [4]:
df_ad = convertToCategoryType(['ip', 'app', 'device', 'os', 'channel'], df_ad)

In [5]:
df_download = df_ad[df_ad.is_attributed == 1]

In [6]:
df_notDownload = df_ad[df_ad.is_attributed != 1].sample(len(df_download)*2)

In [7]:
len(df_ad) - len(df_download)

184447044

In [8]:
df_download.shape

(456846, 8)

In [9]:
df_notDownload.shape

(913692, 8)

In [10]:
df_sample = df_notDownload.append(df_download, ignore_index=True)

In [11]:
df_sample.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,85010,3,1,19,280,2017-11-09 01:11:48,NaN,0
1,85329,3,1,70,424,2017-11-09 13:07:06,NaN,0
2,40436,9,1,19,466,2017-11-08 07:36:30,NaN,0
3,204367,3,1,6,205,2017-11-07 14:12:00,NaN,0
4,82927,1,1,19,13,2017-11-09 08:13:30,NaN,0


In [12]:
def convertDF(df_old):
    df_old['click_time'] = pd.to_datetime(df_old['click_time'], format='%Y/%m/%d %H:%M:%S')
    df_old['hour']=df_old['click_time'].dt.hour
    df_old['date']=df_old['click_time'].dt.date
    df_old['day']=df_old['click_time'].dt.day
    df_old['month']=df_old['click_time'].dt.month
    df_old['year']=df_old['click_time'].dt.year
    return df_old

In [13]:
df_new = convertDF(df_sample)
df_new.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,hour,date,day,month,year
0,85010,3,1,19,280,2017-11-09 01:11:48,NaN,0,1,2017-11-09,9,11,2017
1,85329,3,1,70,424,2017-11-09 13:07:06,NaN,0,13,2017-11-09,9,11,2017
2,40436,9,1,19,466,2017-11-08 07:36:30,NaN,0,7,2017-11-08,8,11,2017
3,204367,3,1,6,205,2017-11-07 14:12:00,NaN,0,14,2017-11-07,7,11,2017
4,82927,1,1,19,13,2017-11-09 08:13:30,NaN,0,8,2017-11-09,9,11,2017


In [14]:
with open('all_converted_data.pkl', 'wb') as picklefile:
    pickle.dump(df_new, picklefile)

In [15]:
df_new.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,hour,date,day,month,year
0,85010,3,1,19,280,2017-11-09 01:11:48,NaN,0,1,2017-11-09,9,11,2017
1,85329,3,1,70,424,2017-11-09 13:07:06,NaN,0,13,2017-11-09,9,11,2017
2,40436,9,1,19,466,2017-11-08 07:36:30,NaN,0,7,2017-11-08,8,11,2017
3,204367,3,1,6,205,2017-11-07 14:12:00,NaN,0,14,2017-11-07,7,11,2017
4,82927,1,1,19,13,2017-11-09 08:13:30,NaN,0,8,2017-11-09,9,11,2017
